In [6]:
#importing the required modules
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import os
from getpass import getpass

In [8]:
my_pass=getpass()

········


In [9]:
#creating the driver instance
s=Service('/home/talha/Softwares/chromedriver_linux64/chromedriver')
myoptions = Options()
driver = Chrome(service=s, options=myoptions)
driver.maximize_window()

#getting the twitter login page
driver.get('https://twitter.com/login')
driver.implicitly_wait(30)

#giving login credentials
time.sleep(2)
username = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//input[@name="text"]'))) 
username.send_keys('TalhaMalik1357')
username.send_keys(Keys.ENTER) 
time.sleep(2)
passwd = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, '//input[@name="password"]')))
passwd.send_keys(my_pass) # actual passwd is saved in an environment variable :)
passwd.send_keys(Keys.ENTER)

#getting tweets to be scrapped
time.sleep(2)
search_input = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH,'//input[@aria-label="Search query"]')))
search_input.send_keys("Shah Mehmood Qureshi")
time.sleep(2)
search_input.send_keys(Keys.ENTER)

#locating to people tag to get shah mehmood qureshi's account in the list of accounts
time.sleep(2)
people = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.LINK_TEXT, 'People')))
people.click()


#getting shah sab's official page opened
time.sleep(2)
get_shah_jee = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[2]/div/section/div/div/div[1]/div/div/div/div/div[2]/div[1]/div[1]/div/div[1]/a/div/div[1]/span/span')))
get_shah_jee.click()

In [10]:
#lets now scrap the tweets just scrapping 50 tweets for sample as i dont have to use this data set for my nlp task
articles = []
tweet_list= []
time_list=[]

while True:
    time.sleep(1)
    article = driver.find_elements(By.TAG_NAME,'article')
    for a in article:
        if a not in articles:
            tweet = a.find_element(By.XPATH, './/*[@data-testid="tweetText"]')
            articles.append(a)
            t = a.find_element(By.XPATH,'.//time')
            time_list.append(t.text)
            tweet_list.append(tweet.text)
    if len(tweet_list) >=50:
        break
    driver.execute_script("window.scrollBy(0,500);")   

In [11]:
#writing the scrapped tweets and thier time to the dataframe

data = {'Time':time_list,'Tweet':tweet_list}
df = pd.DataFrame(data, columns=['Time','Tweet'])
df

#sending data from dataframe to a csv file 
df.to_csv('tweets.csv', index=True)
df = pd.read_csv('tweets.csv')